In [89]:
import numpy as np
from collections import deque
mat = np.array([[0, 1, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 1, 1, 0]])

expected_dist = np.array([[0,1,1,2,3,3,3],
                          [1,0,1,1,2,2,2],
                          [1,1,0,2,3,3,3],
                          [2,1,2,0,1,1,1],
                          [3,2,3,1,0,2,1],
                          [3,2,3,1,2,0,1],
                          [3,2,3,1,1,1,0]])

In [90]:
num_vertices = mat.shape[0]
res = np.full((num_vertices, num_vertices), np.inf)

# Finish this loop
for i in range(num_vertices):
    Q = deque()
    # initialize visited vector
    visited = np.full((num_vertices), False)
    
    # set diagonal elements to distance of 0
    np.fill_diagonal(res,0)
    
    # set first vertex in queue and set as visited
    Q.append([i, 0])
    visited[i] = True

    # run through network while there are still vertices left in the queue
    while Q:
        # current reference distance
        distance = Q[0][1]

        # determine adjacent vertices
        adj_vert = np.where(mat[Q[0][0]] > 0)[0]
        
        # run if there exists any adjacent vertices
        if adj_vert.any():
            
            # filter out vertices that have already been visited
            adj_vert = [vert for vert in adj_vert if visited[vert] == False]
            
            # add adjacent vertices to queue and set them as visited
            [Q.append([vert,distance + 1]) for vert in adj_vert]
            np.put(visited, adj_vert, True)

            # record the distance to the vertices
            np.put(res[i], adj_vert,distance+1)
            
            # remove current vertex from queue
            Q.popleft()

print(res)

[[ 0.  1.  1. inf inf inf inf]
 [ 1.  0.  1. inf inf inf inf]
 [ 1.  1.  0. inf inf inf inf]
 [inf inf inf  0.  1.  1.  1.]
 [inf inf inf  1.  0.  2.  1.]
 [inf inf inf  1.  2.  0.  1.]
 [inf inf inf  1.  1.  1.  0.]]


In [98]:
dist_mat = res
num_vertices = mat.shape[0]
available = [True for _ in range(num_vertices)]
expected_components = [np.array([0,1,2]), np.array([3,4,5,6])]

components = []

# finish this loop
while any(available):
    current_node = available.index(True)

    components.append(np.where(dist_mat[current_node] < np.inf)[0].tolist())
    available[current_node] = False

components = np.unique(components)
print(components)
print(expected_components)

[list([0, 1, 2]) list([3, 4, 5, 6])]
[array([0, 1, 2]), array([3, 4, 5, 6])]
